# Config & Imports

In [3]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


import os

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization

import utilities_LR

from joblib import Parallel, delayed

from IPython.display import Image


In [13]:
config = {
    'data': {
        'n_datasets': 15_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 20, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 10,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0.01,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },    
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae', #tf.keras.losses.get(config['lambda_net']['loss_lambda']),
            'metrics': ['mae']
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 150,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 120,
        'use_gpu': False,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

## Settings

In [14]:

os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices


In [15]:
import logging
logging.getLogger('tensorflow').disabled = True

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [7]:
X_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_features']])

if  config['data']['n_targets'] < 2:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], ])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], ])
else:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_targets']])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], config['data']['n_targets']])

In [8]:
directory = utilities_LR.data_path_LR(config)

with open(directory + '/X_datasets_list_dataForLambda.npy', "rb") as f:
    X_datasets_list = np.load(f, allow_pickle=True)
with open(directory + '/y_datasets_list_dataForLambda.npy', "rb") as f:
    y_datasets_list = np.load(f, allow_pickle=True)
#with open(directory + '/coef_list_targetForInet.npy', "rb") as f:
#    coef_list = np.load(f, allow_pickle=True)

# Prepare Data (Functions)

In [9]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['lambda']['data_prep']['train_test_val_split']['val_size'] / (config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

# Save Model & Metrics (functions)

In [10]:
def save_models(weights_list):
    directory = utilities_LR.lambda_path_LR(config)
    
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    with open(directory + '/lambda_weights_list.npy', "wb") as f:
        np.save(f, weights_list, allow_pickle=True)

# Train Model

In [11]:
def train_nn(X, y, index):
    # Data Prep
    X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                          y)
    
    # Model Def
    model = Sequential()
    model.add(BatchNormalization(input_dim=config['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config['data']['n_targets'], activation='sigmoid'))

    
    model.compile(optimizer=config['lambda']['model_compile']['optimizer_lambda'],
                  loss=config['lambda']['model_compile']['loss'],
                  metrics=config['lambda']['model_compile']['metrics']
                 )
    
    # Model fit
    history = model.fit(x=X_train,
                        y=y_train,
                        batch_size=config['lambda']['model_fit']['batch_size'],
                        epochs=config['lambda']['model_fit']['epochs'],
                        verbose=config['lambda']['model_fit']['verbose'],
                        callbacks=config['lambda']['model_fit']['callbacks'],
                        validation_data=(X_val, y_val),
                        shuffle=config['lambda']['model_fit']['shuffle'],
                        class_weight=config['lambda']['model_fit']['class_weight'],
                        sample_weight=config['lambda']['model_fit']['sample_weight'],
                        initial_epoch=config['lambda']['model_fit']['initial_epoch'],
                        steps_per_epoch=config['lambda']['model_fit']['steps_per_epoch'],
                        validation_steps=config['lambda']['model_fit']['validation_steps'],
                        validation_batch_size=config['lambda']['model_fit']['validation_batch_size'],
                        validation_freq=config['lambda']['model_fit']['validation_freq'],
                       )
    
    
    
    return np.concatenate([x.flatten() for x in model.get_weights()])

In [ ]:
parallel = Parallel(n_jobs=config['computation']['n_jobs'], verbose=10, backend='loky') #loky

weights_list = parallel(delayed(train_nn)(X_data, y_data, index) for index, (X_data, y_data) in enumerate(zip(X_datasets_list, y_datasets_list)))
                                  
del parallel

[Parallel(n_jobs=120)]: Using backend LokyBackend with 120 concurrent workers.
2022-06-07 20:33:57.540981: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-07 20:33:57.541074: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: dws-student-01
2022-06-07 20:33:57.541087: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: dws-student-01
2022-06-07 20:33:57.541361: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.60.2
2022-06-07 20:33:57.541399: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.60.2
2022-06-07 20:33:57.541405: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.60.2
2022-06-07 20:33:57.541780: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized

# Inspect Metrics

In [ ]:
weights_list = np.stack([np.array(x) for x in weights_list])

In [ ]:
weights_list.shape

# Save Models

In [ ]:
save_models(weights_list)

# Create Coefficients for Inet y-Data

In [ ]:
def generate_base_model(config):
    
    model = Sequential()
    model.add(BatchNormalization(input_dim=config['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config['data']['n_targets'], activation='sigmoid'))
    
    return model
    
def shape_flat_network_parameters(flat_network_parameters, target_network_parameters):
               
    shaped_network_parameters =[]
    start = 0  
    
    for parameters in target_network_parameters:
        target_shape = parameters.shape
        size = np.prod(target_shape)
        shaped_parameters = np.reshape(flat_network_parameters[start:start+size], target_shape)
        shaped_network_parameters.append(shaped_parameters)
        start += size

    return shaped_network_parameters

def network_parameters_to_network(network_parameters, config):
    
    model = generate_base_model(config)    

    model_network_parameters = model.get_weights()    
 

    # Shape weights (flat) into correct model structure
    shaped_network_parameters = shape_flat_network_parameters(network_parameters, model_network_parameters)
    
    model.set_weights(shaped_network_parameters)
    
    model.compile(optimizer=config['lambda']['model_compile']['optimizer_lambda'],
                  loss=config['lambda']['model_compile']['loss'],
                  metrics=config['lambda']['model_compile']['metrics']
                 )
    
    return model

In [ ]:
def get_LR(X, y):
    model = LogisticRegression(penalty='l2',
        dual=False,
        tol=0.0001,
        C=1.0,
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=None,
        random_state=None,
        solver='lbfgs',
        max_iter=100,
        multi_class='auto',
        verbose=0,
        warm_start=False,
        n_jobs=None,
        l1_ratio=None
                              )
    model.fit(X, y)
    return model

In [ ]:
def save_coefs(coef_list):
    directory = utilities_LR.lambda_path_LR(config)
    
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    with open(directory + '/lambda_generated_coef_list_target_for_inet.npy', "wb") as f:
        np.save(f, coef_list, allow_pickle=True)

In [ ]:
def create_coef(X_dataset, weights):
    lambda_model =  network_parameters_to_network(weights, config)
    y_data = lambda_model.predict(X_dataset)
    
    y_data = [1.0 if y>=0.5 else 0.0 for y in y_data]
    
    logregmodel = get_LR(X_dataset, y_data)
    
    return logregmodel.coef_

In [ ]:
parallel = Parallel(n_jobs=config['computation']['n_jobs'], verbose=10, backend='loky') #loky

coef_list = parallel(delayed(create_coef)(X_data, weights) for (X_data, weights) in zip(X_datasets_list, weights_list))
                                  
del parallel

In [ ]:
coef_list = np.stack([np.array(x[0]) for x in coef_list])

In [ ]:
coef_list.shape

In [ ]:
save_coefs(coef_list)